In [1]:
import pandas as pd
%matplotlib inline
import mpld3
mpld3.enable_notebook()
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime, date

# Clean, Transform, Save Session data:
## Clean

* 1) Eliminate sessions that have 0 kWh Energy

* 2) Eliminate sessions where total session time is less than 120 seconds (probably driver never really charged his car)

* 3) Ignore sessions with “Connector Type” -  Type 2 Cable & Type 2 Socket

* 4) “Session Type” - FLEET should ideally be analyzed/treated separately since they are not your regular/passenger EV drivers

* 5) Ignore sessions with Energy(kWh) greater than 100 kWh for “Session Type” OTHER

## Transform Datetimes

* transform start time string to datetime, create new column "start_datetime"
* create new columns: "start_" + ["seconds", "year", "month", "day", "weekday"]
* ignore endtime, because it is given by start time + duration
* drop original start and end columns


## Optional Clean:

* 6) Ignore sessions not in PDT/PST timezone

* 7) discard drivers from out of country

* 8) discard sessions with Fees not in USD




In [2]:
# do verbose sanity checks, and do not save in the end
test = False
# working on EC2 instance
cloud = True
# transform datetimes
transform_datetimes = True

In [3]:
# Loading data into pandas dataframe
if cloud:
    path = "../../data/"
    file = "sessions.csv"
else:
    path = "/home/ourownstory/github/data/SLAC"
    file = "Charging_Session_SLAC.csv"

if test:
    nrows = 1e6
else:
    nrows = None
    
def get_csv():
    print("Reading {} rows of dataframe".format(nrows if nrows is not None else "ALL"))
    return pd.read_csv(path + file, index_col=False, nrows=nrows)#, low_memory=False)

In [4]:
df_raw = get_csv()
if test:
    df = df_raw.copy(deep=True)

Reading ALL rows of dataframe


In [5]:
# List all column headers
if test:
    print("Size of CSV: ", len(df_raw))
    print("Columns: ", list(df_raw))

In [6]:
# df.head(5)

# Helper Fun

In [7]:
def remove_less_than(data, col, threshold):
    print("removing {} occurences of {} of less than {}".format(sum(data[col] < threshold), col, threshold))
    return data[data[col] >= threshold]

## Clean 1 - remove sessions with less than 0.1 kWh:

In [8]:
if test:
    s = sum(df['Energy (kWh)'] < 0.1)
    print("number of sessions to be removed: ", s)

In [9]:
# def session_data_remove_zero_energy(data):
#     """remove sessions with 0 kWh"""
#     print("remove sessions with 0 kWh")
#     data_clean = data[data['Energy (kWh)']!=0]
#     print("Number of values removed: ", len(data)-len(data_clean))
#     return data_clean

In [10]:
if test:
#     df_clean_1 = session_data_remove_zero_energy(df)
    df_clean_1 = remove_less_than(df, col='Energy (kWh)', threshold=0.1)


## Clean 2 - remove sessions with Session Time < 2min

In [11]:
if test:
    s = sum(df_clean_1['Session Time (secs)'] < 120)
    print("Number of sessions less than 120 secs long: ", s)

In [12]:
# def session_data_remove_session_time(data, session_time=120):
#     """Removing sessions with Session Time < 2min"""
#     print("Removing sessions with Session Time < 2min")
#     data_clean = data[data['Session Time (secs)'] >= 120]
#     print("Number of values removed: ", len(data) - len(data_clean))
#     return data_clean

In [13]:
if test:
#     df_clean_2 = session_data_remove_session_time(df_clean_1)
    df_clean_2 = remove_less_than(df_clean_1, col='Session Time (secs)', threshold=120)

## Clean 3 - Remove sessions with “Connector Type” - Type 2 Cable & Type 2 Socket

In [14]:
if test:
    session_connector_type = df_clean_2['Connector Type']

    # Types of sessions:
    print("Types of connectors: ", session_connector_type.unique())
    
    # bad connectors
    s =sum((session_connector_type == 'Type 2 Cable') | (session_connector_type == 'Type 2 Socket'))
    if s != 0:
        print("Number of sessions with connectors Type 2: ", s)
    else:
        print("No sessions with Type 2 Connector")  

In [15]:
def session_data_remove_connector(data, conn1='Type 2 Cable', conn2 = 'Type 2 Socket'):
    """Remove sessions with “Connector Type” - Type 2 Cable & Type 2 Socket"""
    print("Remove sessions with “Connector Type” - Type 2 Cable & Type 2 Socket")
    data_clean = data[~((data['Connector Type'] == conn1) | (data['Connector Type'] == conn2))]
    print("Number of values removed: ", len(data) - len(data_clean))
    return data_clean

In [16]:
if test:
    df_clean_3 = session_data_remove_connector(df_clean_2)

## Clean 4 - separating fleet from non fleet

In [17]:
if test:
    session_type = df_clean_3['Session Type']

    # Types of sessions:
    print("Types of sessions: ", session_type.unique())

    s = sum(session_type == "FLEET")

    if s != 0:
        print("Number of sessions that corresponds to fleet: ", s)
    else:
        print("No sessions with fleet data type")

In [18]:
def session_data_split_fleet(data, session_type_name="FLEET"):
    """separate fleet from non fleet"""
    data_clean = data[data['Session Type'] != session_type_name]
    data_clean_fleet = data[data['Session Type'] == session_type_name]
    print("Split into {} NON-{} and {} {}".format(
        len(data_clean), session_type_name, len(data_clean_fleet), session_type_name))
    return data_clean, data_clean_fleet

In [19]:
if test:
    df_clean_4, df_clean_4_fleet = session_data_split_fleet(df_clean_3)

## Clean 5 - ignore sessions with Energy(kWh) greater than 100 kWh for “Session Type” OTHER


In [20]:
if test:
    session_type_other = df_clean_4[df_clean_4['Session Type'] == 'OTHER']
    session_kwh_other = session_type_other['Energy (kWh)']

    s = sum(session_kwh_other > 100)
    print("Number of sessions greater than 100kWh: ", s)

In [21]:
def session_data_other_limit_kwh(data, session_type_name="OTHER", max_kwh=100):
    """ ignore sessions with Energy(kWh) greater than 100 kWh for “Session Type” OTHER"""
    is_other = data['Session Type'] == session_type_name
    is_over_max = data['Energy (kWh)'] > 100
    data_clean = data[~(is_other & is_over_max)]
    print("removed {} sessions with Energy(kWh) greater than 100 kWh for “Session Type” OTHER".format(len(data) - len(data_clean)))
    return data_clean

In [22]:
if test:
    df_clean_5 = session_data_other_limit_kwh(df_clean_4)
    df_clean_5_fleet = session_data_other_limit_kwh(df_clean_4_fleet)

# Transform Datetimes (make new variables)

In [23]:
def to_datetime(x):
    return datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")

def to_seconds(x):
    """seconds in current day since midnight"""
    return 60*60*x.hour + 60*x.minute + x.second

def to_year(x):
    return x.year

def to_month(x):
    return x.month

def to_day(x):
    """day of year, [1, 366]"""
    return x.timetuple().tm_yday

def to_weekday(x):
    """weekday, [0, 6]"""
    return x.weekday()

In [24]:
def apply_transforms(data, col, transforms, names, drop_col=False):
    """apply multiple transforms"""
    for n, t in zip(names, transforms):
        print("applying transfrom {} to {}".format(n, col))
        data[n] = data[col].apply(t)
    if drop_col:
        print("dropping {}".format(col))
        data = data.drop(columns=[col])  
    return data

### handle datetimes: START
apply multiple transforms

In [25]:
def transform_start_datetimes(data):
    col = "Station Start Time (Local)"
    
    # remove sessions where timestamp is not a string
    data_clean = data[data[col].apply(lambda x: type(x)) == type("")]
    print("removed {} sessions where timestamp was not a string".format(len(data) - len(data_clean)))
    
    transforms = [to_datetime, to_seconds, to_year, to_month, to_day, to_weekday]
    names = ["datetime", "seconds", "year", "month", "day", "weekday"]
    names = ["start_{}".format(x) for x in names]
    
    # create datetime object from string
    data_clean = apply_transforms(data_clean, col, [transforms[0]], [names[0]], drop_col=False)
    # all other transfroms are applied to datetime object
    data_clean = apply_transforms(data_clean, names[0], transforms[1:], names[1:], drop_col=False)
    
    # check
#     for n in names:
#         print(df_clean_5[n].head(1))
    
    return data_clean

In [26]:
if test:
    col = "Station Start Time (Local)"
    print(df_clean_5[col][0])
    print(datetime.strptime(df_clean_5[col][0], "%Y-%m-%d %H:%M:%S.%f"))
    print(datetime.strptime(df_clean_5[col][0], "%Y-%m-%d %H:%M:%S.%f"))
    print(df_clean_5[col].apply(lambda x: type(x)).value_counts())
    
    df_clean_5 = transform_start_datetimes(df_clean_5)
    df_clean_5 = df_clean_5.drop(columns=["Station Start Time (Local)"]) 

### handle datetimes: E
remove - it is redundant. 
(start + session time = end time)

In [27]:
if test:
    df_clean_5 = df_clean_5.drop(columns=["Station End Time (Local)"]) 

# Optional Cleaning
Note: only continued for non-fleet

## Helper function: Remove all rows that are not legit values for a column

In [28]:
def remove_out_of(data, col, legit_values, keep_nan=True):
    """Helper function: Remove all rows that are not legit values for a column"""
    print("There are {} NAN entries in column {}".format(data[col].isnull().sum(), col))
    legit = data[col].apply(lambda x: x in legit_values)
    is_nan = keep_nan & data[col].isnull()
    data_clean = data[legit | is_nan]
    print("Removed {} entries for variable {} of values {}".format(
        len(data)-len(data_clean), col, set(data[col].unique()) - set(legit_values + [np.nan])))
    if not keep_nan:
        print("dropped nans")
    return data_clean

## Optional Clean 6 - ignore sessions not in PDT/PST timezone

Note: Is only small fraction


In [29]:
if test:
    column = "Session Time Zone"
    print(df_clean_5[column].value_counts())
    # print(len(sessions[column].unique()))

    # check: if all in California, USA, must be PDT/PST
    col = "Country"
    print(col)
    print(df_clean_5[col].value_counts())
    col = "State"
    print(col)
    print(df_clean_5[col].value_counts())

In [30]:
if test and transform_datetimes:
    # plot START UDT, to sanity check graph form
    df_plot = df_clean_5[df_clean_5["Session Time Zone"].apply(lambda x: x in ["PST", "PDT"])]
    df_plot = df_plot["start_seconds"]
    plot = plt.hist(df_plot/(60*60), bins=50)

In [31]:
if test and transform_datetimes: 
    # plot START non-UT - shifted or random? seems like a mix
    df_plot = df_clean_5[df_clean_5["Session Time Zone"].apply(lambda x: x not in ["PST", "PDT"])]
    df_plot = df_plot["start_seconds"]
    plot = plt.hist(df_plot/(60*60), bins=50)

In [32]:
if test:
    df_clean_6 = remove_out_of(df_clean_5, col="Session Time Zone", legit_values=['PDT', 'PST'], keep_nan=True)

## Optional Clean 7 - discard drivers from out of country
Because they are only here temporarily

Is only small fraction

Keep drivers from other states, because they might just not have changed their info in the database

In [33]:
if test:
    df_clean_7 = remove_out_of(df_clean_6, col="Driver Country", legit_values=["United States"], keep_nan=True)
    # Keep out of state drivers for now
    # df_clean_7 = remove_out_of(df_clean_7, col="Driver State", legit_values=["California"], keep_nan=True)

## Optional Clean 8 -  discard sessions with Fees not in USD
is only small fraction

In [34]:
if test:
    df_clean_8 = remove_out_of(df_clean_7, col="Fee Currency", legit_values=["USD"], keep_nan=True)

# Put all together in one function

In [35]:
def clean_session_data(df, optional=True, transform_datetimes=True):
    """
    * 1) Eliminate sessions that have 0 kWh Energy
    * 2) Eliminate sessions where total session time is less than 120 seconds (probably driver never really charged his car)
    * 3) Ignore sessions with “Connector Type” -  Type 2 Cable & Type 2 Socket
    * 4) “Session Type” - FLEET should ideally be analyzed/treated separately since they are not your regular/passenger EV drivers
    * 5) Ignore sessions with Energy(kWh) greater than 100 kWh for “Session Type” OTHER
    """
    df_clean_1 = remove_less_than(df, col='Energy (kWh)', threshold=0.1)    
    df_clean_2 = remove_less_than(df_clean_1, col='Session Time (secs)', threshold=120)
    df_clean_3 = session_data_remove_connector(df_clean_2)
    df_clean_4, df_clean_4_fleet = session_data_split_fleet(df_clean_3)
    df_clean = session_data_other_limit_kwh(df_clean_4)
    df_clean_fleet = session_data_other_limit_kwh(df_clean_4_fleet)
    
    if transform_datetimes:
        df_clean = transform_start_datetimes(df_clean)
        df_clean = df_clean.drop(columns=["Station Start Time (Local)"]) 
        df_clean = df_clean.drop(columns=["Station End Time (Local)"])         
    
    if optional:
        df_clean_6 = remove_out_of(df_clean, col="Session Time Zone", legit_values=['PDT', 'PST'], keep_nan=True)
        df_clean_7 = remove_out_of(df_clean_6, col="Driver Country", legit_values=["United States"], keep_nan=True)
        # Keep out of state drivers for now
        # df_clean_7 = remove_out_of(df_clean_7, col="Driver State", legit_values=["California"], keep_nan=True)
        df_clean = remove_out_of(df_clean_7, col="Fee Currency", legit_values=["USD"], keep_nan=True) 
        
    return df_clean, df_clean_fleet

# Clean, Transform and Save Data

In [36]:
if not test:
    # clean, transform
    df_clean, df_clean_fleet = clean_session_data(df_raw, optional=True, transform_datetimes=True)
    print("change in size:",  len(df_clean)/len(df_raw))
    
    # save
    path += "clean/"
    name = 'sessions_clean.csv'
    print("saving: ", name)
    df_clean.to_csv(path + name, sep=',', index=False)
    name = 'sessions_clean_fleet.csv'
    print("saving: ", name)
    df_clean_fleet.to_csv(path + name, sep=',', index=False)
    

removing 496730 occurences of Energy (kWh) of less than 0.1
removing 7613 occurences of Session Time (secs) of less than 120
Remove sessions with “Connector Type” - Type 2 Cable & Type 2 Socket
Number of values removed:  108
Split into 11749580 NON-FLEET and 188801 FLEET
removed 228 sessions with Energy(kWh) greater than 100 kWh for “Session Type” OTHER
removed 0 sessions with Energy(kWh) greater than 100 kWh for “Session Type” OTHER
removed 0 sessions where timestamp was not a string
applying transfrom start_datetime to Station Start Time (Local)
applying transfrom start_seconds to start_datetime
applying transfrom start_year to start_datetime
applying transfrom start_month to start_datetime
applying transfrom start_day to start_datetime
applying transfrom start_weekday to start_datetime
There are 0 NAN entries in column Session Time Zone
Removed 26368 entries for variable Session Time Zone of values {'UTC', 'EDT', 'MST', 'EST', 'MHT', 'MDT', 'IST'}
There are 146872 NAN entries in col